論文  
https://arxiv.org/abs/2112.05131  
  
GitHub  
https://github.com/sxyu/svox2  
  
<a href="https://colab.research.google.com/github/kaz12tech/ai_demos/blob/master/Plenoxels_demo.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ランタイムの設定
「ランタイム」→「ランタイムのタイプを変更」→「ハードウェアアクセラレータ」をGPUに変更

# 実行方法
「ランタイム」→「すべてのセルを実行」を選択

# GPU確認

In [ ]:
!nvidia-smi

# GoogleDriveのマウント

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# GitHubからPlenoxelsのソースコードを取得

In [ ]:
%cd /content/

!git clone https://github.com/sxyu/svox2.git

In [ ]:
!sed -E -i "s/\{minv=:/minv=\{minv:/g"    /content/svox2/opt/opt.py 
!sed -E -i "s/\{meanv=:/meanv=\{meanv:/g" /content/svox2/opt/opt.py 
!sed -E -i "s/\{maxv=:/maxv=\{maxv:/g"    /content/svox2/opt/opt.py 
!sed -E -i "s/\{minv=:/minv=\{minv:/g"    /content/svox2/opt/render_imgs.py 
!sed -E -i "s/\{meanv=:/meanv=\{meanv:/g" /content/svox2/opt/render_imgs.py 
!sed -E -i "s/\{maxv=:/maxv=\{maxv:/g"    /content/svox2/opt/render_imgs.py 

# ライブラリのインストール

In [ ]:
%cd /content/svox2
!apt install ninja-build -y
!pip install imageio-ffmpeg
!pip install ipdb
!pip install lpips
!pip install pymcubes

!pip install .

# ライブラリのインポート

In [ ]:
import os
from moviepy.editor import *
from moviepy.video.fx.resize import resize

# 学習済みモデルのダウンロード
gdownを使用してダウンロードできないため  
https://drive.google.com/drive/folders/1SOEJDw8mot7kf5viUK9XryOAmZGe_vvE のckpt_syn.tar.gzを右クリックし「ドライブにショートカットを追加」を選択し、マイドライブを選択後、「ショートカットを追加」  
自身のGoogle Driveにショートカットを作成  

In [ ]:
%cd /content/svox2
!mkdir ckpt
%cd ckpt

if not os.path.exists("/content/svox2/ckpt/ckpt_syn.tar.gz"):
  !cp /content/drive/MyDrive/ckpt_syn.tar.gz /content/svox2/ckpt
!tar -xvf ./ckpt_syn.tar.gz > /dev/null

# データセットのダウンロード
gdownを使用してダウンロードできないため  
https://drive.google.com/drive/folders/128yBriW1IG_3NJ5Rp7APSTZsJqdJdfc1 のnerf_synthetic.zipを右クリックし「ドライブにショートカットを追加」を選択し、マイドライブを選択後、「ショートカットを追加」  
自身のGoogle Driveにショートカットを作成  

In [ ]:
%cd /content/svox2/
!mkdir datasets
%cd datasets

if not os.path.exists("/content/svox2/datasets/nerf_synthetic.zip"):
  !cp /content/drive/MyDrive/nerf_synthetic.zip /content/svox2/datasets/
!unzip ./nerf_synthetic.zip > /dev/null

# 自由視点画像の生成

In [ ]:
%cd /content/svox2/
!mkdir -p /content/svox2/results/hotdog
!cp /content/svox2/ckpt/256_to_512_fasttv/hotdog/ckpt.npz /content/svox2/results/hotdog

In [ ]:
%%time

%cd /content/svox2/opt

!python render_imgs.py /content/svox2/results/hotdog/ckpt.npz /content/svox2/datasets/nerf_synthetic/hotdog/

## 自由視点画像の表示

In [ ]:
%cd /content/svox2/

results_video = "/content/svox2/results/hotdog/test_renders.mp4"
clip = VideoFileClip(results_video)
clip = resize(clip, height=420)
clip.ipython_display()

In [ ]:
%cd /content/svox2/
!mkdir -p /content/svox2/results/materials
!cp /content/svox2/ckpt/256_to_512_fasttv/materials/ckpt.npz /content/svox2/results/materials

In [ ]:
%%time

%cd /content/svox2/opt

!python render_imgs_circle.py /content/svox2/results/materials/ckpt.npz /content/svox2/datasets/nerf_synthetic/materials/

In [ ]:
%cd /content/svox2/

results_video = "/content/svox2/results/materials/circle_renders.mp4"
clip = VideoFileClip(results_video)
clip = resize(clip, height=420)
clip.ipython_display()